In [2]:
import requests, time
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from collections import Counter
import functools
import operator
import string
from sklearn.model_selection import train_test_split

In [3]:
def get_categories():
    url = 'https://clientsfromhell.net/'
    html = requests.get(url)
    response = bs(html.content)
    get_items = [category for category in response.find_all('li', {'class':'flex items-center'})]
    categories = ['Dunces','Criminals','Deadbeats','Racists','Homophobes','Sexist','Frenemies','Cryptic','Ingrates','Chaotic Good']
    category_pair = []
    for item in get_items:
        href = item.find('a').get('href')
        item_name = re.sub('\\n','',item.text)
        pair = (item_name, href)
        if item_name in categories:
            category_pair.append(pair)
    return list(set(category_pair))

In [4]:
list_catetories = get_categories()
list_catetories

[('Deadbeats', '/tag/payday-mayday/'),
 ('Frenemies', '/tag/Old-Friend/'),
 ('Ingrates', '/tag/Live-to-work/'),
 ('Cryptic', '/tag/Clear-as-coal/'),
 ('Homophobes', '/tag/Heterosapien/'),
 ('Chaotic Good', '/tag/hell-michigan/'),
 ('Racists', '/tag/Unwinnable-race/'),
 ('Sexist', '/tag/Breasts-and-Brains/'),
 ('Dunces', '/tag/ignoramus/'),
 ('Criminals', '/tag/freelance-felon/')]

In [5]:
list_url_patters = []

def URL_CATEGORY_CREATOR(x):
    for cat in list_catetories:
        pattern = 'https://clientsfromhell.net'+cat[1]+'page/' # regex pattern for the urls to scrape
        list_url_patters.append((pattern,cat[0]))
    return list_url_patters
        
url_cat = URL_CATEGORY_CREATOR(list_catetories)
url_cat

[('https://clientsfromhell.net/tag/payday-mayday/page/', 'Deadbeats'),
 ('https://clientsfromhell.net/tag/Old-Friend/page/', 'Frenemies'),
 ('https://clientsfromhell.net/tag/Live-to-work/page/', 'Ingrates'),
 ('https://clientsfromhell.net/tag/Clear-as-coal/page/', 'Cryptic'),
 ('https://clientsfromhell.net/tag/Heterosapien/page/', 'Homophobes'),
 ('https://clientsfromhell.net/tag/hell-michigan/page/', 'Chaotic Good'),
 ('https://clientsfromhell.net/tag/Unwinnable-race/page/', 'Racists'),
 ('https://clientsfromhell.net/tag/Breasts-and-Brains/page/', 'Sexist'),
 ('https://clientsfromhell.net/tag/ignoramus/page/', 'Dunces'),
 ('https://clientsfromhell.net/tag/freelance-felon/page/', 'Criminals')]

In [6]:
for url in url_cat:
    print(url[0])

https://clientsfromhell.net/tag/payday-mayday/page/
https://clientsfromhell.net/tag/Old-Friend/page/
https://clientsfromhell.net/tag/Live-to-work/page/
https://clientsfromhell.net/tag/Clear-as-coal/page/
https://clientsfromhell.net/tag/Heterosapien/page/
https://clientsfromhell.net/tag/hell-michigan/page/
https://clientsfromhell.net/tag/Unwinnable-race/page/
https://clientsfromhell.net/tag/Breasts-and-Brains/page/
https://clientsfromhell.net/tag/ignoramus/page/
https://clientsfromhell.net/tag/freelance-felon/page/


In [7]:
def PAGE_NUM_CREATOR(x):
    list_url_num =[]
    for url in url_cat:
        html = requests.get(url[0]+'1')
        response = bs(html.content)
        list_items = response.find_all('a',{'class':'page-numbers'})

        len_=len(list_items)-2
        max_pag=list_items[len_].text
        list_url_num.append((url[0],max_pag,url[1]))
    return list_url_num

list_all = PAGE_NUM_CREATOR(url_cat)
list_all

[('https://clientsfromhell.net/tag/payday-mayday/page/', '58', 'Deadbeats'),
 ('https://clientsfromhell.net/tag/Old-Friend/page/', '4', 'Frenemies'),
 ('https://clientsfromhell.net/tag/Live-to-work/page/', '24', 'Ingrates'),
 ('https://clientsfromhell.net/tag/Clear-as-coal/page/', '51', 'Cryptic'),
 ('https://clientsfromhell.net/tag/Heterosapien/page/', '3', 'Homophobes'),
 ('https://clientsfromhell.net/tag/hell-michigan/page/', '40', 'Chaotic Good'),
 ('https://clientsfromhell.net/tag/Unwinnable-race/page/', '6', 'Racists'),
 ('https://clientsfromhell.net/tag/Breasts-and-Brains/page/', '6', 'Sexist'),
 ('https://clientsfromhell.net/tag/ignoramus/page/', '82', 'Dunces'),
 ('https://clientsfromhell.net/tag/freelance-felon/page/', '14', 'Criminals')]

In [8]:
class IronhackSpider:
    
    def __init__(self, url_pattern, pages_to_scrape=1, sleep_interval=-1, content_parser=None):
        self.url_pattern = url_pattern
        self.pages_to_scrape = pages_to_scrape
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
  
    def scrape_url(self, url):
        response = requests.get(url)
        result = self.content_parser(response.content)
        return result
#         self.list_results(result)
        
#     def list_results(self, r):
#         list_pages = []
#         list_pages.append(r)
#         return list_pages

#     def output_results(self, r):
#         print(r)

#     def kickstart(self):
#         for i in range(1, self.pages_to_scrape+1):
#             self.scrape_url(self.url_pattern % i)
            
    def kickstart(self):
        list_pages = []
        for i in range(1, self.pages_to_scrape+1):
            list_pages.append(self.scrape_url(self.url_pattern % i))            
        return list_pages


def content_parser(content):
    return content

def case_parser(content):
    all_content = bs(content, "html.parser")
    pre_content = all_content.select('div [class="w-blog-post-content"] > p')
    
    case=[]
    
    for i, el in enumerate(pre_content):
        text = el.text
        case.append(text)

    return case

# Instantiate the IronhackSpider class
html_cont_dict = {}

for URL_PATTERN, PAGES_TO_SCRAPE, CAT in list_all:
#     print(URL_PATTERN, PAGES_TO_SCRAPE, CAT)
    my_spider = IronhackSpider(URL_PATTERN+'%s/', int(PAGES_TO_SCRAPE), content_parser=case_parser)
    # Start scraping jobs
    content = my_spider.kickstart()
    html_cont_dict.update({CAT: content})
    
#     html_cont_dict.update(CAT, my_spider.kickstart())
    
html_cont_dict['Frenemies']

[['One of my clients referred me to his friend for a new website for his retail store.',
  'I met him at his store and we discussed what he needs and explained everything for almost two hours.',
  'When I gave him an estimate of how much it will cost(very generous) he almost starts shouting.',
  'Client: I am not paying you this much.',
  "Me: Well, for the amount of work involved this is best I can do. You want an eCommerce store that supports 600 products that has to integrate with your POS system. It's a big job.\xa0",
  "Client:\xa0But my wife told me I can make a website with WIX for 20 bucks. You seem like a good guy so I'll pay you 100 dollars, but I need a website to be ready in a week.",
  'I tried my best but had to say goodbye.',
  'I am a graphic design student and a few days ago I designed a resume for myself. It was in full color and featured infographics; I figured it should be eye-catching.',
  'Later, I showed it to a friend who wanted me to do one like it for him. ',


In [9]:
import csv
with open('client_from_hell_dic.csv', 'w') as f:
    for key in html_cont_dict.keys():
        f.write("%s,%s\n"%(key,html_cont_dict[key]))

In [10]:
stopwords = ["mustn't", 'have', 'now', "shouldn't", 'down', 'theirs', 
             'where', 'doing', 'who', 'is', 'how', 'which', 'up', 'if', 
             'nor', 'y', 'a', 'couldn', "weren't", 'your', 'in', 'themselves', 
             'only', 'both', 'will', 'own', 'we', 'by', 'these', 'into', 'once', 
             'her', 'when', 'o', 's', 've', 'again', 'mustn', 'they', 't', 'aren', 
             'his', "haven't", "won't", 'same', 'not', 'there', 'hadn', 'has', 
             'can', "didn't", 'those', 'then', 'this', 'shan', 'such', 'wasn', 
             'of', 'after', 'through', 'from', "you'd", 'shouldn', "isn't", 'being', 
             'too', "that'll", 'between', 'had', 'yours', 'very', 'i', 'because', 
             'him', 'are', "needn't", 'before', 'wouldn', 'my', 'what', 'the', 'during', 
             'isn', 'weren', 'against', 'don', 'off', 'll', 'am', 'our', 'should', 
             "should've", "you've", 'hers', 'their', 'more', 'than', 'himself', 'itself', 
             'with', 'just', "hasn't", 'he', 'any', 'to', "couldn't", "you'll", 'on', 'she', 
             "she's", 'ma', 'an', 'haven', 'was', 'few', 'each', 'that', 'further', 'didn', 
             "hadn't", "it's", 'over', 'you', 'me', 'about', 'mightn', 'most', 'be', 'so', 
             'won', 'them', 'why', 'needn', 'or', 'above', "doesn't", 'd', 'yourselves', 
             'ours', 'herself', 'as', 'some', 'under', "don't", 'at', 'all', "wouldn't", 'it', 
             'does', 'but', 'below', 'here', 'other', 'whom', 'ourselves', "you're", 'did', 'do', 
             'while', "aren't", 'having', 're', 'ain', 'were', 'hasn', 'no', 'out', 'and', 'for', 
             'yourself', 'been', 'm', 'myself', 'its', 'until', "shan't", "wasn't", "mightn't", 'doesn', 
             "you’re", "aren’t"]

test = "you're"

test not in stopwords

False

In [11]:
df_clients_og = pd.DataFrame.from_dict(html_cont_dict, orient = 'index').fillna('')
df_clients_og = df_clients_og.T

def stem(a):
    p = PorterStemmer()
    a = [p.stem(word) for word in a]
    return a

# df_clients_og = pd.DataFrame.from_dict(html_cont_dict, orient = 'index').fillna('')
# df_clients_og = df_clients_og.T

for col in df_clients_og:
#     print(col)
        
    for i,list_ in enumerate(df_clients_og[col]):
        sub_list=[]

        for item in list_:
            if item.startswith('Client:'):
                sub_list.append(item)

        df_clients_og[col][i] = sub_list
#         print(sub_list)
        
punc_list = [x for x in string.punctuation]

for col in df_clients_og:
#     print(col)
        
    for i,list_ in enumerate(df_clients_og[col]):
        sub_list = [x.replace('\xa0', ' ') for x in df_clients_og[col][i]]
        sub_list = [x.replace('\n', ' ') for x in sub_list]
        sub_list = [x.replace('Client: ', '') for x in sub_list]
        for punc in punc_list:
            sub_list = [x.replace(punc, '') for x in sub_list]
        sub_list = [x.replace('—', '') for x in sub_list]
        sub_list = [x.replace('   ', ' ') for x in sub_list]
        sub_list = [x.replace('  ', ' ') for x in sub_list]
        sub_list = [x.rstrip() for x in sub_list]
        
        df_clients_og[col][i] = sub_list
        
for col in df_clients_og:
#     print(col)
        
    for i,list_ in enumerate(df_clients_og[col]):
        sub_list = [x.split(' ') for x in list_]
        
        df_clients_og[col][i] = sub_list
        df_clients_og[col][i] = [word.lower() for words in df_clients_og[col][i] for word in words if len(word) != 1]
        df_clients_og[col][i] = [re.sub(r'^(.)\1+', r'\1', word)  for word in df_clients_og[col][i]]

        df_clients_og[col][i] = [word.replace("’", "'") for word in df_clients_og[col][i]]
#         df_clients_og[col][i] = [word.replace("ing", "") for word in df_clients_og[col][i]]
#         df_clients_og[col][i] = [word.replace("ed", "") for word in df_clients_og[col][i]]
        df_clients_og[col][i] = [word.rstrip("'") for word in df_clients_og[col][i]]
        
        
        
        df_clients_og[col][i] = [word for word in df_clients_og[col][i] if word not in stopwords]
        df_clients_og[col][i] = [word for word in df_clients_og[col][i] if word.isalpha() == True]
        df_clients_og[col][i] = [word for word in df_clients_og[col][i] if len(word) != 1]
        df_clients_og[col][i] = stem(df_clients_og[col][i])
        
        print(df_clients_og[col][i])
        
        df_clients_og[col][i] = dict(Counter(df_clients_og[col][i]))

['got', 'bill', 'say', 'owe', 'send', 'bill', 'septemb', 'yall', 'well', 'check', 'account', 'onlin', 'said', 'zero', 'balanc', 'well', 'gonna', 'pay', 'pay', 'tomorrow', 'ye', 'want', 'pay', 'stock', 'imageri', 'use', 'origin', 'work', 'way', 'pay', 'extra', 'danc', 'wellknown', 'danc', 'school', 'like', 'make', 'dress', 'look', 'forward', 'dress', 'love', 'design', 'much', 'hi', 'want', 'ask', 'dress', 'finish', 'deposit', 'get', 'start', 'pay', 'whole', 'dress', 'know', 'student', 'favour', 'great', 'exampl', 'art', 'cours', 'kind', 'money', 'bad', 'person', 'take', 'advantag', 'broke', 'student', 'knock', 'price', 'make', 'quicker', 'pleas', 'proceed', 'banner', 'go', 'send', 'proof', 'pay', 'alway', 'gotten', 'proof', 'pay', 'absolut', 'need', 'friday', 'pleas', 'hold', 'thing', 'payment', 'think', 'get', 'anim', 'art', 'redon', 'program', 'websit', 'mayb', 'visit', 'next', 'week', 'could', 'talk', 'cool', 'theoret', 'long', 'would', 'take', 'much', 'wow', 'young', 'alreadi', 'eye

['think', 'half', 'amount', 'three', 'month', 'prove', 'worth', 'obvious', 'surpris', 'oh', 'realli', 'hmmm', 'absolut', 'love', 'work', 'honestli', 'bit', 'concern', 'amount', 'time', 'invest', 'project', 'time', 'cost', 'without', 'person', 'profession', 'insult', 'like', 'ask', 'consid', 'bill', 'us', 'half', 'invest', 'time', 'use', 'produc', 'brochur', 'great', 'afraid', 'offer', 'small', 'stipend', 'work', 'need', 'design', 'print', 'magazin', 'think', 'take', 'hour', 'labor', 'pay', 'total', 'took', 'look', 'work', 'realli', 'like', 'saw', 'look', 'mani', 'connect', 'promot', 'throughout', 'citi', 'send', 'event', 'shoot', 'problem', 'pay', 'photograph', 'togeth', 'chang', 'event', 'work', 'chang', 'need', 'outdoor', 'shot', 'campaign', 'simpl', 'eleg', 'shot', 'three', 'model', 'nice', 'outdoor', 'locat', 'regard', 'onlin', 'catalog', 'need', 'item', 'photograph', 'everi', 'angl', 'creat', 'anim', 'us', 'photoshop', 'want', 'shot', 'front', 'green', 'screen', 'add', 'differ', '

['pleas', 'sign', 'new', 'contract', 'sent', 'ident', 'last', 'contract', 'sign', 'chang', 'show', 'new', 'compani', 'name', 'need', 'start', 'work', 'afternoon', 'alreadi', 'miss', 'critic', 'deadlin', 'op', 'sorri', 'sent', 'contract', 'anoth', 'develop', 'ignor', 'work', 'last', 'contract', 'sign', 'nice', 'tri', 'pay', 'transfer', 'know', 'budget', 'tight', 'lost', 'money', 'produc', 'event', 'enough', 'money', 'pay', 'fee', 'take', 'fee', 'compens', 'spearhead', 'project', 'us', 'lose', 'money', 'wish', 'well', 'come', 'pay', 'much', 'lot', 'expens', 'thought', 'would', 'thought', 'start', 'would', 'ask', 'sever', 'reduc', 'suppos', 'know', 'would', 'stick', 'ye', 'like', 'logo', 'sure', 'think', 'compar', 'could', 'send', 'us', 'ep', 'hd', 'format', 'help', 'us', 'think']
['know', 'easi', 'set', 'use', 'wordpress', 'coupl', 'year', 'ago', 'need', 'add', 'coupl', 'plugin', 'copi', 'content', 'take', 'effort', 'skill', 'frankli', 'think', 'free', 'least', 'come', 'fine', 'look', 'w

['need', 'someon', 'improv', 'seo', 'busi', 'work', 'websit', 'still', 'long', 'way', 'go', 'would', 'improv', 'seo', 'see', 'yeah', 'need', 'improv', 'thing', 'well', 'let', 'know', 'want', 'bring', 'board', 'need', 'two', 'thing', 'realli', 'quit', 'urgent', 'wed', 'appreci', 'swift', 'respons', 'weve', 'two', 'design', 'let', 'us', 'recent', 'realli', 'behind', 'schedul', 'love', 'idea', 'need', 'discuss', 'intern', 'well', 'touch', 'brilliant', 'work', 'hi', 'still', 'keen', 'board', 'avail', 'let', 'know', 'asap', 'use', 'bee', 'readi', 'photograph', 'bring', 'clay', 'took', 'hour', 'make', 'bee', 'gonna', 'take', 'forev', 'need', 'work', 'studio', 'sinc', 'need', 'much', 'guidanc', 'pay', 'quot', 'took', 'long', 'bug', 'clay', 'photoshop', 'rest', 'draw', 'much', 'would', 'charg', 'one', 'bug', 'way', 'much', 'well', 'gonna', 'discuss', 'come', 'tomorrow', 'hey', 'real', 'quick', 'weve', 'secur', 'artist', 'use', 'poster', 'art', 'theyll', 'send', 'copi', 'artwork', 'soon', 'fini

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['hi', 'hire', 'friend', 'logo', 'us', 'strong', 'fix', 'logo', 'want', 'show', 'ribbon', 'bubbl', 'actual', 'bubbl', 'look', 'like', 'soap', 'way', 'bubbl', 'like', 'fine', 'bubbl', 'small', 'someth', 'one', 'last', 'one', 'ribbon', 'squar', 'enough', 'end', 'know', 'job', 'figur', 'design', 'choic', 'turn', 'centuri', 'font', 'vertic', 'soonest', 'see', 'problem', 'societi', 'work', 'homeless', 'peopl', 'beg', 'money', 'compani', 'like', 'clearli', 'need', 'help', 'guy', 'someth', 'well', 'realli', 'sure', 'get', 'point', 'much', 'issss', 'remov', 'issss', 'sound', 'remov', 'issss', 'sound', 'cannot', 'use', 'issss', 'use', 'unaccept', 'still', 'issss', 'remov', 'sound', 'like', 'amateur', 'issss', 'thought', 'profession', 'studio', 'remov', 'issss', 'demand', 'refund', 'design', 'birthday', 'tag', 'candi', 'like', 'one', 'want', 'tag', 'much', 'dont', 'know', 'make', 'like', 'one', 

...

['avail', 'work', 'invit', 'gala', 'ask', 'look', 'realli', 'want', 'want', 'time', 'sorri', 'make', 'video', 'shorter', 'realli', 'time', 'deal', 'figur', 'someth', 'need', 'draw', 'someth', 'fanci', 'simpl', 'draw', 'howev', 'like', 'matter', 'tell', 'later', 'fine', 'make', 'thing', 'difficult', 'doctor', 'make', 'video', 'clip', 'audio', 'file', 'need', 'visual', 'make', 'video', 'file', 'one', 'simpli', 'take', 'imag', 'internet', 'copyright', 'make', 'use', 'problem', 'guess', 'could', 'ye', 'good', 'busi', 'person', 'start', 'wirefram', 'get', 'layout', 'right', 'worri', 'style', 'later', 'corpor', 'color', 'wast', 'time', 'even', 'get', 'brand', 'right', 'matter', 'work', 'matter', 'import', 'god', 'told', 'hello', 'need', 'sign', 'door', 'full', 'color', 'door', 'full', 'color', 'door', 'full', 'color', 'door', 'alreadi', 'holder', 'mount', 'door', 'everyth', 'show', 'pictur', 'took', 'cell', 'phone', 'help', 'oh', 'god', 'see', 'hard', 'need', 'sign', 'door', 'full', 'color']

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['offic', 'door', 'littl', 'girl', 'work', 'offens', 'thing', 'call', 'well', 'call', 'cute', 'appenin', 'lad', 'seen', 'ye', 'messi', 'call', 'mean', 'messag', 'ye', 'av', 'wall', 'lad', 'happi', 'aba', 'photo', 'yano', 'hire', 'photograph', 'get', 'pictur', 'bird', 'rest', 'great', 'well', 'ask', 'yo', 'focu', 'bird', 'know', 'wer', 'sayin', 'us', 'askin', 'approach', 'properli', 'compli', 'tbh', 'look', 'like', 'shite', 'parti', 'pictur', 'scatti', 'kid', 'wall', 'wall', 'birdsss', 'got', 'even', 'use', 'photo', 'look', 'like', 'twelv', 'nineyearold', 'granddaught', 'size', 'rais', 'son', 'alon', 'well', 'would', 'marri', 'heart', 'product', 'men', 'model', 'nude', 'believ', 'unprofession', 'never', 'work', 'keep', 'stubborn', 'never', 'real', 'model', 'attitud', 'know', 'work', 'exposur', 'start', 'move', 'video', 'chat', 'video', 'ch

['want', 'exact', 'blue', 'third', 'book', 'left', 'shelf', 'avoid', 'salt', 'see', 'salt', 'made', 'sodium', 'chlorid', 'highli', 'volatil', 'chemic', 'hello', 'need', 'help', 'log', 'backend', 'server', 'know', 'usernam', 'work', 'ye', 'sure', 'well', 'work', 'need', 'fix', 'thought', 'email', 'root', 'usernam', 'usernam', 'like', 'domain', 'name', 'tell', 'usernam', 'stop', 'play', 'game', 'help', 'usernam', 'go', 'help', 'least', 'repeat', 'block', 'peopl', 'leav', 'bad', 'review', 'wow', 'printer', 'better', 'mine', 'oh', 'sorri', 'sent', 'full', 'list', 'includ', 'intern', 'item', 'rather', 'ecommerc', 'stock', 'sure']
['right', 'well', 'compani', 'call', 'name', 'match', 'initi', 'know', 'tell', 'take', 'second', 'display', 'page', 'precalcul', 'everi', 'possibl', 'permut', 'across', 'everi', 'project', 'simpli', 'write', 'page', 'reflect', 'state', 'instead', 'calcul', 'everi', 'time', 'display', 'page', 'match', 'patron', 'web', 'page', 'bit', 'hard', 'drive', 'actual', 'physi

['yeah', 'spywar', 'purpos', 'simpl', 'groundbreak', 'idea', 'know', 'nobodi', 'thought', 'tell', 'make', 'half', 'site', 'area', 'rectangular', 'blank', 'space', 'fill', 'later', 'tell', 'steal', 'idea', 'simpl', 'need', 'teach', 'work', 'done', 'publish', 'ok', 'tell', 'need', 'complet', 'silenc', 'done', 'okay', 'simpl', 'need', 'client', 'abl', 'see', 'go', 'dress', 'make', 'onlin', 'want', 'liter', 'see', 'told', 'simpl', 'program', 'part', 'monitor', 'mirror', 'like', 'new', 'teacher', 'tall', 'deal', 'kid', 'hair', 'long', 'regist', 'blaketheelevatorgriffincom', 'need', 'sell', 'thing', 'make', 'money', 'nah', 'googl', 'search', 'blake', 'elev', 'griffin', 'got', 'million', 'hit', 'feel', 'like', 'entitl', 'least', 'dollar', 'hit', 'send', 'logo', 'jpg', 'jpeg', 'nice', 'tri', 'stupid', 'hell', 'happen', 'flyer', 'yeah', 'like', 'one', 'new', 'one', 'horribl', 'one', 'compani', 'sent', 'morn', 'look', 'pretti', 'good', 'need', 'chang', 'font', 'offici', 'univers', 'font', 'remem

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [12]:
all_keys = []

for col in df_clients_og:
#     print(col)
        
    for i,dic_ in enumerate(df_clients_og[col]):
        for word in list(dic_.keys()):
            all_keys.append(word)

all_keys = list(set(all_keys))

# print(len(all_keys))
    
all_keys = sorted(all_keys)
# print(all_keys)
        

In [13]:
df_final = df_clients_og.T

df_final.columns = [str(col) for col in df_final.columns]

df_final.reset_index(inplace = True)
df_final.rename(columns = {'index':'category'}, inplace = True)
# df_final.head()

df_cases = pd.DataFrame(columns = ['category', 'case'])

# df_final[['category', '0']].rename(columns = {'0':'case'})

for col in df_final:
    if col != 'category':
        df_cases = df_cases.append(df_final[['category', col]].rename(columns = {col:'case'}))

df_cases.reset_index(drop = True, inplace = True)

for i, row in enumerate(df_cases['case']):
    if row == {}:
        df_cases.drop(index = i, inplace = True)
        
df_cases.reset_index(drop = True, inplace = True)
df_cases


,category,case
0,Deadbeats,"{'got': 1, 'bill': 2, 'say': 1, 'owe': 1, 'sen..."
1,Frenemies,"{'pay': 3, 'much': 2, 'wife': 1, 'told': 1, 'm..."
2,Ingrates,"{'go': 3, 'focu': 1, 'around': 2, 'programm': ..."
3,Cryptic,"{'hi': 1, 'hire': 1, 'friend': 1, 'logo': 3, '..."
4,Homophobes,"{'loud': 1, 'proud': 1, 'gay': 1, 'understand'..."
...,...,...
283,Dunces,"{'name': 1, 'sort': 1, 'stupid': 1, 'introduc'..."
284,Dunces,"{'finish': 3, 'product': 2, 'seem': 1, 'exactl..."
285,Dunces,"{'decid': 2, 'want': 1, 'call': 1, 'new': 1, '..."
286,Dunces,"{'outrag': 1, 'last': 1, 'crew': 1, 'could': 1..."


In [14]:
df_count = pd.DataFrame()

for i in range(df_cases['case'].shape[0]):
#     print(i)
    df_count = df_count.append(pd.DataFrame(df_cases['case'][i], index=[0]))
    
df_count.reset_index(drop = True, inplace = True)
df_count.head()

# df_count.fillna('0', inplace = True)


# df_count = pd.DataFrame(df_cases['case'][0], index=[0])

# df_count.append(pd.DataFrame(df_cases['case'][1], index=[0]))

# df_count

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,aba,abandon,abil,abl,abomin,aboutpag,abroad,absent,absolut,abstract,...,youv,yup,yur,zero,zine,zip,zlist,zone,zoom,zoomin
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_count.shape[0]

286

In [16]:
df_count['category'] = df_cases['category']

In [17]:
df_count.head()

,aba,abandon,abil,abl,abomin,aboutpag,abroad,absent,absolut,abstract,...,yup,yur,zero,zine,zip,zlist,zone,zoom,zoomin,category
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homophobes
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Ingrates
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Racists
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cryptic
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chaotic Good


In [18]:
df_count.to_csv('final_dataframe.csv')

In [19]:
df_final = pd.read_csv('final_dataframe.csv')

In [20]:
df_final.drop(columns = 'Unnamed: 0', inplace = True)

In [21]:
df_new_final = df_final.fillna('0')

# df_final.head()

In [22]:
features = df_new_final[[col for col in df_new_final.columns if col != 'category']]
features = features.astype(int)

In [23]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Columns: 3849 entries, aba to zoomin
dtypes: int64(3849)
memory usage: 8.4 MB


In [24]:
target = df_new_final[['category']]
target

for i in range(target.shape[0]):
    target['category'] = [x.replace(x, '1') if x == 'Deadbeats' else x.replace(x,'0') for x in target['category']]
    
target = target.astype(int)
# type(target['category'][100])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [25]:
target['category'][0]

0

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    features, # Features (X)
    target, # Target (y)
    test_size = .2,
    random_state = 42
)

X_train.head()

,aba,abandon,abil,abl,abomin,aboutpag,abroad,absent,absolut,abstract,...,youv,yup,yur,zero,zine,zip,zlist,zone,zoom,zoomin
139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
82,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
y_train.head()

,category
139,0
25,0
82,0
144,0
66,0


In [28]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3).fit(X_train, y_train.values.ravel())

knn.score(X_test, y_test)

1.0

In [29]:
from sklearn.naive_bayes import BernoulliNB

bernoulli = BernoulliNB().fit(X_train, y_train.values.ravel())
bernoulli.score(X_test, y_test)

1.0

In [30]:
from sklearn.naive_bayes import GaussianNB

guassian = GaussianNB().fit(X_train, y_train.values.ravel())
guassian.score(X_test, y_test)

1.0

In [135]:
def model_score_mul(x):
    import time
    
    score = x.score(X_test, y_test_mul)*100
    if score >= 50.0:
        print('The score for this algorithm is (drum roll please!!!): ')
        time.sleep(3)
        print('.... processing')
        time.sleep(3)
        print('....')
        time.sleep(3)
        print('*looks at you confused*')
        time.sleep(3)
        print('oh yeah, you are waiting for the score... LOL')
        print('score: ',score,'%')
        print("DON'T GET COCKY NOW!!! KEEP MAKING IT BETTER!")
    elif score < 50.0:
        print('I got good news, and bad news...')
        time.sleep(2)
        print('the bad news is that you will not like the score...')
        time.sleep(2)
        print("Now, the good news is that I'm right here to watch your face!!! LOL")
        print('The score for this algorithm is: ',score,'%')
        time.sleep(2)
        print('')
        print("Your algorithm stinks so much, I could toss a coin and make better predictions =P...")

In [97]:
cat_dic = {'Dunces': 0,'Criminals': 1,'Deadbeats': 2,'Racists': 3,'Homophobes': 4,
        'Sexist': 5,'Frenemies': 6,'Cryptic': 7,'Ingrates': 8,'Chaotic Good': 9}

In [143]:
target_mul = df_new_final[['category']].replace(cat_dic, regex=True)
# target_mul.head()

# df['commentTest'] = df['Comment'].replace(ids, regex=True)

X_train, X_test, y_train_mul, y_test_mul = train_test_split(
    features, # Features (X)
    target_mul, # Target (y)
    test_size = .2,
    random_state = 42
)

from sklearn.linear_model import LogisticRegression

log_regr = LogisticRegression(solver = 'lbfgs')
log_regr.fit(X_train, y_train_mul.values.ravel())
model_score_mul(log_regr)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


The score for this algorithm is (drum roll please!!!): 
.... processing
....
*looks at you confused*
oh yeah, you are waiting for the score... LOL
score:  65.51724137931035 %
DON'T GET COCKY NOW!!! KEEP MAKING IT BETTER!


In [136]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3) # k = 5 by default
knn.fit(X_train, y_train_mul.values.ravel())
model_score_mul(knn)

I got good news, and bad news...
the bad news is that you will not like the score...
Now, the good news is that I'm right here to watch your face!!! LOL
The score for this algorithm is:  41.37931034482759 %

Your algorithm stinks so much, I could toss a coin and make better predictions =P...


In [100]:
from sklearn.preprocessing import RobustScaler

scalar = RobustScaler().fit(X_train)
scalar.transform(X_train)[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [137]:
knn_robust = KNeighborsClassifier()
knn_robust.fit(scalar.transform(X_train), y_train_mul.values.ravel())
model_score_mul(knn_robust)

I got good news, and bad news...
the bad news is that you will not like the score...
Now, the good news is that I'm right here to watch your face!!! LOL
The score for this algorithm is:  37.93103448275862 %

Your algorithm stinks so much, I could toss a coin and make better predictions =P...


In [102]:
from sklearn.naive_bayes import MultinomialNB

multi = MultinomialNB()
multi.fit(X_train, y_train_mul.values.ravel())
model_score_mul(multi)

The score for this algorithm is (drum roll please!!!): 
.... processing
....
*looks at you confused*
oh yeah, you are waiting for the score... LOL
score:  56.896551724137936 %
DON'T GET COCKY NOW!!! KEEP MAKING IT BETTER!


In [103]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=10, random_state=42)
rfc.fit(X_train, y_train_mul.values.ravel())
model_score_mul(rfc)

The score for this algorithm is:  44.827586206896555 %
Tough luck buddy... try something else "/


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [104]:
features_bool = df_new_final[[col for col in df_new_final.columns if col != 'category']]
features_bool = features.astype(bool)

In [105]:
features_bool.head()

,aba,abandon,abil,abl,abomin,aboutpag,abroad,absent,absolut,abstract,...,youv,yup,yur,zero,zine,zip,zlist,zone,zoom,zoomin
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [106]:
X_train_bool, X_test_bool, y_train_mul, y_test_mul = train_test_split(
    features_bool, # Features (X)
    target_mul, # Target (y)
    test_size = .2,
    random_state = 42
)

X_train_bool.head()

,aba,abandon,abil,abl,abomin,aboutpag,abroad,absent,absolut,abstract,...,youv,yup,yur,zero,zine,zip,zlist,zone,zoom,zoomin
139,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
82,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
144,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
66,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False


In [107]:
y_train_mul

,category
139,7
25,1
82,1
144,7
66,2
...,...
188,2
71,0
106,0
270,0


In [108]:
bernoulli = BernoulliNB().fit(X_train_bool, y_train_mul.values.ravel())
model_score_mul(bernoulli)

The score for this algorithm is (drum roll please!!!): 
.... processing
....
*looks at you confused*
oh yeah, you are waiting for the score... LOL
score:  55.172413793103445 %
DON'T GET COCKY NOW!!! KEEP MAKING IT BETTER!


In [109]:
pred_log_regr = log_regr.predict(X_test)
pred_multi = multi.predict(X_test)
pred_bernoulli = bernoulli.predict(X_test_bool)


In [110]:
from sklearn.metrics import r2_score

print('R^2 for Log Regression: ', r2_score(y_test_mul, pred_log_regr))
print('R^2 for Multinomial NB: ', r2_score(y_test_mul, pred_multi))
print('R^2 for Bernoulli NB: ', r2_score(y_test_mul, pred_bernoulli))


R^2 for Log Regression:  -0.09819963257807718
R^2 for Multinomial NB:  -0.39796693202694433
R^2 for Bernoulli NB:  -0.7645070422535212


In [111]:
from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(linear['actual'], linear['predicted'])

print('MSE for Log Regression: ', mean_squared_error(y_test_mul, pred_log_regr))
print('MSE for Multinomial NB: ', mean_squared_error(y_test_mul, pred_multi))
print('MSE for Bernoulli NB: ', mean_squared_error(y_test_mul, pred_bernoulli))


MSE for Log Regression:  13.327586206896552
MSE for Multinomial NB:  16.96551724137931
MSE for Bernoulli NB:  21.413793103448278


In [112]:
from math import sqrt

print('RMSE for Log Regression: ', sqrt(mean_squared_error(y_test_mul, pred_log_regr)))
print('RMSE for Multinomial NB: ', sqrt(mean_squared_error(y_test_mul, pred_multi)))
print('RMSE for Bernoulli NB: ', sqrt(mean_squared_error(y_test_mul, pred_bernoulli)))

RMSE for Log Regression:  3.650696674183785
RMSE for Multinomial NB:  4.118921854245272
RMSE for Bernoulli NB:  4.627503982002422


In [113]:
from sklearn.metrics import mean_absolute_error

# mean_absolute_error(linear['actual'], linear['predicted'])

print('MAE for Log Regression: ', mean_absolute_error(y_test_mul, pred_log_regr))
print('MAE for Multinomial NB: ', mean_absolute_error(y_test_mul, pred_multi))
print('MAE for Bernoulli NB: ', mean_absolute_error(y_test_mul, pred_bernoulli))

MAE for Log Regression:  1.9137931034482758
MAE for Multinomial NB:  2.4827586206896552
MAE for Bernoulli NB:  2.896551724137931


In [114]:
from sklearn.metrics import accuracy_score

# accuracy_score(classifier['actual'], classifier['predicted'])

print('Accuracy score for Log Regression: ', accuracy_score(y_test_mul, pred_log_regr))
print('Accuracy score for Multinomial NB: ', accuracy_score(y_test_mul, pred_multi))
print('Accuracy score for Bernoulli NB: ', accuracy_score(y_test_mul, pred_bernoulli))

Accuracy score for Log Regression:  0.6551724137931034
Accuracy score for Multinomial NB:  0.5689655172413793
Accuracy score for Bernoulli NB:  0.5517241379310345


In [115]:
from sklearn.metrics import balanced_accuracy_score

# balanced_accuracy_score(classifier['actual'], classifier['predicted'])

print('Balanced accuracy score for Log Regression: ', balanced_accuracy_score(y_test_mul, pred_log_regr))
print('Balanced accuracy score for Multinomial NB: ', balanced_accuracy_score(y_test_mul, pred_multi))
print('Balanced accuracy score for Bernoulli NB: ', balanced_accuracy_score(y_test_mul, pred_bernoulli))

Balanced accuracy score for Log Regression:  0.3103174603174603
Balanced accuracy score for Multinomial NB:  0.24629629629629632
Balanced accuracy score for Bernoulli NB:  0.22777777777777775


In [116]:
inv_cat_dic = dict(map(reversed, cat_dic.items()))
inv_cat_dic

{0: 'Dunces',
 1: 'Criminals',
 2: 'Deadbeats',
 3: 'Racists',
 4: 'Homophobes',
 5: 'Sexist',
 6: 'Frenemies',
 7: 'Cryptic',
 8: 'Ingrates',
 9: 'Chaotic Good'}

In [117]:
from sklearn.metrics import confusion_matrix

confusion = pd.DataFrame(
    data = confusion_matrix(y_test_mul, pred_log_regr),
)

confusion.rename(columns = inv_cat_dic, inplace = True)
confusion.rename(index = inv_cat_dic, inplace = True)

print('Log Regression')
confusion

Log Regression


,Dunces,Criminals,Deadbeats,Racists,Homophobes,Sexist,Frenemies,Cryptic,Ingrates
Dunces,17,0,0,0,0,0,2,0,1
Criminals,0,0,1,0,0,0,0,0,0
Deadbeats,0,0,12,0,0,0,0,0,0
Racists,0,0,0,0,0,0,1,0,0
Homophobes,1,0,0,0,0,0,1,0,0
Sexist,0,0,1,0,0,0,0,0,0
Frenemies,1,1,0,0,0,0,8,0,0
Cryptic,1,0,1,0,0,0,0,0,2
Ingrates,3,0,2,0,0,0,1,0,1


In [119]:
confusion_1 = pd.DataFrame(
    data = confusion_matrix(y_test_mul, pred_multi),
)

confusion_1.rename(columns = inv_cat_dic, inplace = True)
confusion_1.rename(index = inv_cat_dic, inplace = True)

print('Multinomial NB')
confusion_1

Multinomial NB


,Dunces,Criminals,Deadbeats,Racists,Homophobes,Sexist,Frenemies,Cryptic,Ingrates
Dunces,18,0,2,0,0,0,0,0,0
Criminals,0,0,1,0,0,0,0,0,0
Deadbeats,1,0,11,0,0,0,0,0,0
Racists,0,0,0,0,0,0,1,0,0
Homophobes,1,0,0,0,0,0,0,0,1
Sexist,1,0,0,0,0,0,0,0,0
Frenemies,5,0,1,0,0,0,4,0,0
Cryptic,1,0,2,0,0,0,0,0,1
Ingrates,4,0,3,0,0,0,0,0,0


In [120]:
confusion_2 = pd.DataFrame(
    data = confusion_matrix(y_test_mul, pred_bernoulli),
)

confusion_2.rename(columns = inv_cat_dic, inplace = True)
confusion_2.rename(index = inv_cat_dic, inplace = True)

print('Bernoulli NB')
confusion_2

Bernoulli NB


,Dunces,Criminals,Deadbeats,Racists,Homophobes,Sexist,Frenemies,Cryptic,Ingrates
Dunces,20,0,0,0,0,0,0,0,0
Criminals,0,0,1,0,0,0,0,0,0
Deadbeats,3,0,9,0,0,0,0,0,0
Racists,1,0,0,0,0,0,0,0,0
Homophobes,2,0,0,0,0,0,0,0,0
Sexist,1,0,0,0,0,0,0,0,0
Frenemies,7,0,0,0,0,0,3,0,0
Cryptic,4,0,0,0,0,0,0,0,0
Ingrates,6,0,1,0,0,0,0,0,0


In [125]:
from sklearn.metrics import precision_score

precision = precision_score(
    y_test_mul,
    pred_log_regr,
    pos_label = 2,
    average = 'weighted'
)

precision_1 = precision_score(
    y_test_mul,
    pred_multi,
    pos_label = 2,
    average = 'weighted'
)

precision_2 = precision_score(
    y_test_mul,
    pred_bernoulli,
    pos_label = 2,
    average = 'weighted'
)

print('Log Regression precision score: ', precision)
print('Multinomial NB precision score: ', precision_1)
print('Bernulli NB precision score: ', precision_2)

Log Regression precision:  0.5371903980136628
Multinomial NB precision:  0.45194660734149056
Bernulli NB precision:  0.4984326018808777


In [127]:
from sklearn.metrics import recall_score

# recall = recall_score(
#     classifier['actual'],
#     classifier['predicted'],
#     pos_label = 'A'
# )

recall = recall_score(
    y_test_mul,
    pred_log_regr,
    pos_label = 2,
    average = 'weighted'
)

recall_1 = recall_score(
    y_test_mul,
    pred_multi,
    pos_label = 2,
    average = 'weighted'
)

recall_2 = recall_score(
    y_test_mul,
    pred_bernoulli,
    pos_label = 2,
    average = 'weighted'
)

print('Log Regression recall score: ', recall)
print('Multinomial NB recall score: ', recall_1)
print('Bernulli NB recall score: ', recall_2)

Log Regression recall:  0.6551724137931034
Multinomial NB recall:  0.5689655172413793
Bernulli NB recall:  0.5517241379310345


In [128]:
from sklearn.metrics import f1_score

# f1_score(
#     classifier['actual'],
#     classifier['predicted'],
#     pos_label = 'A'
# )

f1_score_0 = f1_score(
    y_test_mul,
    pred_log_regr,
    pos_label = 2,
    average = 'weighted'
)

f1_score_1 = f1_score(
    y_test_mul,
    pred_multi,
    pos_label = 2,
    average = 'weighted'
)

f1_score_2 = f1_score(
    y_test_mul,
    pred_bernoulli,
    pos_label = 2,
    average = 'weighted'
)

print('Log Regression f1 score: ', f1_score_0)
print('Multinomial NB f1 score: ', f1_score_1)
print('Bernulli NB f1 score: ', f1_score_2)

Log Regression f1 score:  0.5857627066032487
Multinomial NB f1 score:  0.47760311020960106
Bernulli NB f1 score:  0.45701187867604653


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [141]:
knn_robust = KNeighborsClassifier()
knn_robust.fit(scalar.transform(X_train), y_train_mul.values.ravel())
model_score_mul(knn_robust)

I got good news, and bad news...
the bad news is that you will not like the score...
Now, the good news is that I'm right here to watch your face!!! LOL
The score for this algorithm is:  37.93103448275862 %

Your algorithm stinks so much, I could toss a coin and make better predictions =P...
